In [ ]:
# Run once

# !ssh-keygen -t rsa -b 4096 -f /content/drive/MyDrive/ML_Sem3_Project/.ssh/id_rsa -N "" -q
# !cat /content/drive/MyDrive/ML_Sem3_Project/.ssh/id_rsa.pub

In [ ]:
# Run everytime

from google.colab import drive
drive.mount('/content/drive')

!mkdir -p ~/.ssh
!cp /content/drive/MyDrive/ML_Sem3_Project/.ssh/id_rsa ~/.ssh/
!cp /content/drive/MyDrive/ML_Sem3_Project/.ssh/id_rsa.pub ~/.ssh/
!chmod 600 ~/.ssh/id_rsa
!ssh-keyscan github.com >> ~/.ssh/known_hosts

import os

repo_dir = "/content/ML_Project_2025"

if not os.path.exists(repo_dir):
  !git clone git@github.com:astroartics/ML_Project_2025.git {repo_dir}
else:
  %cd {repo_dir}
  !git pull

%cd {repo_dir}

Mounted at /content/drive
# github.com:22 SSH-2.0-aae3c6b
# github.com:22 SSH-2.0-aae3c6b
# github.com:22 SSH-2.0-aae3c6b
# github.com:22 SSH-2.0-aae3c6b
# github.com:22 SSH-2.0-aae3c6b
Cloning into '/content/ML_Project_2025'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 173 (delta 92), reused 124 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (173/173), 1.57 MiB | 1.41 MiB/s, done.
Resolving deltas: 100% (92/92), done.
/content/ML_Project_2025


In [ ]:

# Run everytime

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nUsing device:", device, "\n")

checkpoint_path = "/content/drive/MyDrive/ML_Sem3_Project/preprocessed_data.pt"
checkpoint = torch.load(
    checkpoint_path,
    map_location = device,  # ensures safe loading on CPU/GPU
    weights_only = False
)

# --------------------------------

# 4. Extract Variables
vocab = checkpoint["vocab"]
inv_vocab = checkpoint["inv_vocab"]

# Move tensors to device
X_padded = checkpoint["X_padded"].to(device)  # PyTorch does not run natively on TPU, hence using T4 GPU
y_tensor = checkpoint["y_tensor"].to(device)

# DataFrame cannot move to device; it stays on CPU
data = checkpoint["data"]

print(data["moves"].head())

# --------------------------------

# 5. Status
print("\nLoaded checkpoint successfully.\n")
print("X_padded shape:", X_padded.shape)
print("y_tensor shape:", y_tensor.shape)
print("Example vocab size:", len(vocab))


Using device: cuda 

0    e4 d5 Nf3 dxe4 Ne5 Nf6 d4 exd3 Bxd3 e6 Nc3 Bd6...
1    b4 e5 Bb2 d6 c3 Bf5 d3 Nf6 e4 Bg6 Be2 Be7 Nf3 ...
2    e4 d5 exd5 Qxd5 Nc3 Qa5 Nf3 Nf6 d4 Bg4 Bd2 Nc6...
3    e4 e5 Nf3 Nf6 Nc3 d6 Bc4 Be6 Qe2 Nbd7 d4 Bxc4 ...
4    d4 e6 c4 c6 Nc3 d5 cxd5 cxd5 e4 Nc6 e5 Qb6 Nge...
Name: moves, dtype: object

Loaded checkpoint successfully.

X_padded shape: torch.Size([658379, 292])
y_tensor shape: torch.Size([658379])
Example vocab size: 3672


In [ ]:
# Push changes to GitHub

!git config --global user.email "joshisanjanana114@gmail.com"
!git config --global user.name "Sanjana"

!cp /content/drive/MyDrive/ML_Sem3_Project/BaselineLSTM.ipynb /content/ML_Project_2025/

!git add BaselineLSTM.ipynb
!git commit -m "Added Notebook"
!git push

In [ ]:
# Train/test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_padded,
    y_tensor,
    test_size = 0.2,  # 80-20 split
    random_state = 42,
    shuffle=True
)

X_train.shape, X_test.shape


(torch.Size([526703, 292]), torch.Size([131676, 292]))

In [ ]:
# PyTorch Datasets and DataLoaders

# Why these are needed:
'''
1) Data cannot be fed to the model directly, because the number of integers is huge.
2) It must be given to the model in batches.
3) A DataLoader iterates over these batches.
4) DataLoader handles batching, shuffling, and feeding the model (if batch size = 64, then taking 64 sequences for X and 64 labels for y)
5) DataLoader also uses multiple workers to load batches faster.
6) Dataset allows Python to treat the data like a list (shuffling indices, slicing samples, fetching mini-batches automatically)
7) Dataset is a wrapper that tells PyTorch "How to get the ith sample."
8) Dataset: Defines how to access each sample
   DataLoader: Defines how to group examples into batches efficiently during training
'''

from torch.utils.data import Dataset, DataLoader

# X and y are not converted to a new format, they are just wrapped and become accessible through the Dataset interface
# For example, before Dataset, we manually index
# sample_X = X_train[100]
# sample_y = y_train[100]
# but, after Dataset, DataLoader can automatically fetch train_ds[100] -> returns (X_train[100], y_train[100])
class ChessDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    # Returns one sample to the DataLoader when requested: DataLoader needs data to be wrapped in a Dataset to
    # access it one by one randomly to form batches.
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# DataLoaders

batch_size = 64

train_dataset = ChessDataset(X_train, y_train)
test_dataset  = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size = batch_size)


In [ ]:
# LSTM

import torch
import torch.nn as nn

# Defining the Baseline LSTM model
# Training time for 5 epochs using a GPU: ~15 minutes

class BaselineLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, padding_idx=0):
        super().__init__()
        # Embedding: converts token IDs -> dense vectors
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim, padding_idx=padding_idx)
        # LSTM: processes sequence of embeddings
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        # Final linear layer to produce logits for each vocabulary token
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        # x: LongTensor of shape (batch_size, seq_len)
        emb = self.embedding(x)                     # -> (batch_size, seq_len, embed_dim)
        outputs, (h_n, c_n) = self.lstm(emb)        # outputs -> (batch_size, seq_len, hidden_dim)
                                                   # h_n -> (num_layers, batch_size, hidden_dim)
        last_hidden = outputs[:, -1, :]             # take output at last time-step -> (batch_size, hidden_dim)
        logits = self.fc(last_hidden)               # -> (batch_size, vocab_size)
        return logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(vocab)        # vocab from your preprocessing (includes PAD and UNK)
model = BaselineLSTM(vocab_size=vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, padding_idx=vocab['<PAD>']).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=vocab['<PAD>'])   # ignore pad index if used in targets (usually not)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 5
lstm_train_losses = []

model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    for X_batch, y_batch in train_loader:
        # Move to device
        X_batch = X_batch.to(device)                   # (batch_size, seq_len), dtype=torch.long
        y_batch = y_batch.to(device)                   # (batch_size,), dtype=torch.long (class indices)

        optimizer.zero_grad()

        logits = model(X_batch)                        # (batch_size, vocab_size)
        loss = criterion(logits, y_batch)              # CrossEntropy expects (N, C) and targets (N,)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)    # sum-of-losses for averaging later

    avg_loss = total_loss / len(train_loader.dataset)
    lstm_train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{num_epochs}  Train loss: {avg_loss:.4f}")

# Predicting / decoding one example
model.eval()
with torch.no_grad():
    sample_X = X_test[0].unsqueeze(0).to(device)      # shape (1, seq_len)
    logits = model(sample_X)                          # (1, vocab_size)
    pred_id = logits.argmax(dim=-1).item()            # integer token id
    pred_move = inv_vocab[pred_id]                    # decode to SAN using your inv_vocab
    print("Predicted token id:", pred_id, "Predicted SAN:", pred_move)


Epoch 1/5  Train loss: 6.3863
Epoch 2/5  Train loss: 6.3628
Epoch 3/5  Train loss: 6.3591
Epoch 4/5  Train loss: 6.3574


In [10]:
# Transformer
# Training time for 5 epochs using a GPU: ~30 minutes

import math
import torch
import torch.nn as nn

# Since trandformers have no notion of order, we must add positional encoding information
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer("pe", pe)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1)]

class TransformerBaseline(nn.Module):
    def __init__(
        self,
        vocab_size,
        d_model=128,
        nhead=4,
        num_layers=2,
        dim_feedforward=512,
        dropout=0.1,
        padding_idx=0,
        max_len=5000
    ):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=d_model,
            padding_idx=padding_idx
        )

        self.positional_encoding = PositionalEncoding(d_model, max_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True   # IMPORTANT: matches your (B, T, D)
        )

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """

        # Padding mask: True where PAD
        padding_mask = (x == vocab['<PAD>'])

        emb = self.embedding(x)                    # (B, T, D)
        emb = self.positional_encoding(emb)        # (B, T, D)

        out = self.transformer_encoder(
            emb,
            src_key_padding_mask=padding_mask
        )                                           # (B, T, D)

        # Take last non-pad token representation
        lengths = (~padding_mask).sum(dim=1) - 1   # (B,)
        last_hidden = out[torch.arange(out.size(0)), lengths]

        logits = self.fc(last_hidden)               # (B, vocab_size)
        return logits

# Instantiating model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TransformerBaseline(
    vocab_size=len(vocab),
    d_model=128,
    nhead=4,
    num_layers=2,
    dim_feedforward=512,
    padding_idx=vocab['<PAD>']
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop (Same as LSTM)
num_epochs = 5
transformer_train_losses = []

model.train()
for epoch in range(num_epochs):
    total_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()

        logits = model(X_batch)
        loss = criterion(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)

    avg_loss = total_loss / len(train_loader.dataset)
    transformer_train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{num_epochs}  Train loss: {avg_loss:.4f}")

# Predicting the next move
model.eval()
with torch.no_grad():
    sample_X = X_test[0].unsqueeze(0).to(device)
    logits = model(sample_X)
    pred_id = logits.argmax(dim=-1).item()
    pred_move = inv_vocab[pred_id]

    print("Predicted move:", pred_move)

Epoch 1/5  Train loss: 5.8599
Epoch 2/5  Train loss: 5.3223
Epoch 3/5  Train loss: 5.0798
Epoch 4/5  Train loss: 4.9201
Epoch 5/5  Train loss: 4.8062
Predicted move: Nc3


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:515: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


In [ ]:
import matplotlib.pyplot as plt

plt.plot(lstm_train_losses, label="LSTM")
plt.plot(transformer_train_losses, label="Transformer")
plt.xlabel("Epoch")
plt.ylabel("Cross-Entropy Loss")
plt.legend()
plt.title("Training Loss Comparison")
plt.show()

In [11]:
# !pip install python-chess pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 124.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=74f3161223d9aa9a19082d735c96db1c6814d8226f744a3d300f8bbe1c262d43
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [ ]:
# Chess game GUI

import chess
import chess.svg
from IPython.display import SVG, display

def show_board(board):
    display(SVG(chess.svg.board(board=board)))

def encode_move_sequence(move_list, vocab, max_len):
    ids = [vocab.get(m, vocab['<UNK>']) for m in move_list]
    ids = ids[-max_len:]
    ids = [vocab['<PAD>']] * (max_len - len(ids)) + ids
    return torch.tensor(ids, dtype=torch.long)

board = chess.Board()
moves = ["e4", "e5", "Nf3", "Nc6"]

for san in moves:
    board.push_san(san)

show_board(board)

pred_move = predict_next_move(transformer_model, moves)
print("Transformer predicts:", pred_move)

try:
    board.push_san(pred_move)
    show_board(board)
except:
    print("Illegal move predicted")
